In [1]:
from sklearn.cluster import KMeans,MeanShift, estimate_bandwidth , Birch
import numpy as np
import scipy.sparse as sp
import igraph as ig
import leidenalg as la
import pandas as pd
from collections import Counter
from sklearn.metrics import adjusted_rand_score, rand_score,normalized_mutual_info_score
import networkx as nx
import community as community_louvain
from scipy.sparse import csr_matrix
from sklearn.neighbors import NearestNeighbors
import community as community_louvain  

In [2]:
df = pd.read_csv("some-reddit.csv")
df

,rotoreuters,PC1,PC2,PC3,PC4,PC5,PC6,PC7,PC8,PC9,PC10
0,fiplefip,11.809120,-0.236315,9.712246,-3.558550,0.676302,-0.330307,-2.824460,-0.289928,0.484788,-3.428256
1,amici_ursi,11.039474,0.714586,7.176633,-6.500434,1.621503,0.676836,-3.538689,-0.039432,2.002969,0.370842
2,unremovable,12.308345,2.944823,13.467455,-6.039847,-1.338466,-0.693709,-2.730207,-3.215488,-1.463924,0.722347
3,CDRE_64,9.597274,-0.275656,9.219146,-5.203834,-0.475436,0.101921,-3.843878,-0.283923,1.681646,0.044997
4,noeatnosleep,8.441664,-1.520713,2.302667,-4.226753,1.043632,1.490670,-6.036377,0.880610,-0.303432,0.104033
5,PostNationalism,5.013857,-2.501058,3.895118,2.235447,3.307937,-0.196699,1.413836,-2.910550,-0.214350,-0.770090
6,trueworldnews,12.048599,1.202438,6.710602,-3.770383,1.228732,-1.718225,-2.476334,-2.163013,3.820246,1.960525
7,drewiepoodle,8.218326,-2.528193,4.460269,-0.936959,-1.685539,-4.491433,-3.246064,-2.502605,-1.823646,1.836632
8,RIPmod,20.111155,2.807410,19.705938,-5.255570,-0.362161,-4.346634,-0.508715,-7.902928,2.057837,1.270654
9,michaelconfoy,4.567328,-8.470312,8.456068,0.283074,-2.701900,-0.641383,1.128566,1.066758,-4.142665,0.631451


In [3]:
X = df.iloc[:, 1:].values

In [4]:
k = 2

nbrs = NearestNeighbors(n_neighbors=k, metric='cosine').fit(X)
distances, indices = nbrs.kneighbors(X)

In [5]:
rows = np.repeat(np.arange(len(X)), k)
cols = indices.flatten()
vals = 1 - distances.flatten()        # تبدیل distance → similarity

A = sp.csr_matrix((vals, (rows, cols)), shape=(len(X), len(X)))
A

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 98 stored elements and shape (49, 49)>

In [6]:
A = A.maximum(A.T)
A

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 135 stored elements and shape (49, 49)>

In [9]:
import community.community_louvain as community_louvain

In [10]:
similarity_sparse =(A)
G = nx.Graph()
rows, cols = similarity_sparse.nonzero()
for i, j in zip(rows, cols):
    if i < j:  # فقط یک سمت برای گراف غیرجهتی
        G.add_edge(i, j, weight=similarity_sparse[i, j])
partition = community_louvain.best_partition(G, weight='weight')
(partition)

{np.int32(0): 1,
 np.int32(3): 1,
 np.int32(48): 1,
 np.int32(1): 1,
 np.int32(4): 1,
 np.int32(2): 2,
 np.int32(14): 2,
 np.int32(44): 2,
 np.int32(22): 1,
 np.int32(5): 3,
 np.int32(11): 3,
 np.int32(6): 4,
 np.int32(12): 4,
 np.int32(25): 4,
 np.int32(7): 7,
 np.int32(16): 7,
 np.int32(8): 8,
 np.int32(19): 8,
 np.int32(9): 7,
 np.int32(29): 7,
 np.int32(41): 7,
 np.int32(10): 6,
 np.int32(32): 6,
 np.int32(23): 3,
 np.int32(34): 4,
 np.int32(13): 5,
 np.int32(42): 5,
 np.int32(15): 7,
 np.int32(40): 7,
 np.int32(20): 7,
 np.int32(39): 7,
 np.int32(17): 7,
 np.int32(18): 8,
 np.int32(37): 7,
 np.int32(21): 6,
 np.int32(46): 6,
 np.int32(31): 1,
 np.int32(30): 3,
 np.int32(24): 5,
 np.int32(35): 4,
 np.int32(45): 4,
 np.int32(26): 6,
 np.int32(27): 0,
 np.int32(36): 0,
 np.int32(28): 2,
 np.int32(33): 5,
 np.int32(38): 4,
 np.int32(47): 7,
 np.int32(43): 6}

In [17]:
from networkx.algorithms.community.quality import modularity

In [18]:
class EvaluationMetrics:
    def __init__(self, graph=None, partition=None):
        """
        graph: NetworkX Graph
        partition: dictionary {node: community_id}
        """
        self.graph = graph
        self.partition = partition

        # metrics
        self.y_true = None
        self.y_pred = None

        self._purity = None
        self._nmi = None
        self._ari = None
        self._ri = None
        self._modularity = None

    # -------------------------
    # Purity
    # -------------------------
    def _compute_purity(self):
        y_true = np.asarray(self.y_true)
        y_pred = np.asarray(self.y_pred)

        clusters = np.unique(y_pred)
        N = len(y_true)
        total = 0

        for c in clusters:
            idx = np.where(y_pred == c)[0]
            true_labels = y_true[idx]
            if len(true_labels) == 0:
                continue
            most_common = Counter(true_labels).most_common(1)[0][1]
            total += most_common

        return total / N

    # -------------------------
    # MAIN FIT
    # -------------------------
    def fit(self, y_true, y_pred):
        """
        y_true, y_pred should be 1D arrays
        """
        self.y_true = np.asarray(list(y_true))
        self.y_pred = np.asarray(list(y_pred))

        # purity
        self._purity = self._compute_purity()

        # nmi
        self._nmi = normalized_mutual_info_score(self.y_true, self.y_pred)

        # ari
        self._ari = adjusted_rand_score(self.y_true, self.y_pred)

        # ri
        self._ri = rand_score(self.y_true, self.y_pred)

        # modularity (NetworkX)
        if self.graph is not None and self.partition is not None:
            # partition must be list of sets
            communities = {}
            for node, com in self.partition.items():
                communities.setdefault(com, set()).add(node)
            community_list = list(communities.values())

            try:
                self._modularity = modularity(self.graph, community_list, weight="weight")
            except:
                self._modularity = None

        return self

    # -------------------------
    # GETTERS
    # -------------------------
    def score_purity(self):
        return self._purity

    def score_nmi(self):
        return self._nmi

    def score_ari(self):
        return self._ari

    def score_ri(self):
        return self._ri

    def score_modularity(self):
        return self._modularity

In [19]:
x = partition.values()
x

dict_values([1, 1, 1, 1, 1, 2, 2, 2, 1, 3, 3, 4, 4, 4, 7, 7, 8, 8, 7, 7, 7, 6, 6, 3, 4, 5, 5, 7, 7, 7, 7, 7, 8, 7, 6, 6, 1, 3, 5, 4, 4, 6, 0, 0, 2, 5, 4, 7, 6])

In [20]:
metrics = EvaluationMetrics(graph=G, partition=partition)
x = list(partition.values())
metrics.fit(x,x)

print("Purity:", metrics.score_purity())
print("NMI:", metrics.score_nmi())
print("ARI:", metrics.score_ari())
print("RI:", metrics.score_ri())
print("Modularity:", round(metrics.score_modularity(),3))

Purity: 1.0
NMI: 1.0
ARI: 1.0
RI: 1.0
Modularity: 0.778


### معیار های درست 

In [32]:
class CommunityMetrics:
    def __init__(self, graph, partition):
        """
        graph: networkx.Graph
        partition: dict {node: community_id} یا list[list(nodes)]
        """
        self.graph = graph
        
        # اگر partition دیکشنری است، آن را به لیست لیست تبدیل می‌کنیم
        if isinstance(partition, dict):
            communities = {}
            for node, c in partition.items():
                communities.setdefault(c, []).append(node)
            self.communities = list(communities.values())
        else:
            self.communities = partition

    # ----------------------------
    # 1. Modularity (Newman)
    # ----------------------------
    def modularity(self):
        return nx.algorithms.community.quality.modularity(self.graph, self.communities, weight='weight')

    # ----------------------------
    # 2. Conductance
    # ----------------------------
    def conductance(self):
        conductances = []
        for community in self.communities:
            cut_size = nx.algorithms.cut_size(self.graph, community, weight='weight')
            volume = sum([self.graph.degree(n, weight='weight') for n in community])
            if volume == 0:
                continue
            conductances.append(cut_size / volume)
        return np.mean(conductances)

    # ----------------------------
    # 3. Cut Ratio
    # ----------------------------
    def cut_ratio(self):
        n = self.graph.number_of_nodes()
        m = self.graph.number_of_edges()
        total_cut = 0
        for community in self.communities:
            cut_edges = nx.algorithms.cut_size(self.graph, community, weight='weight')
            total_cut += cut_edges
        if n*(n-1)/2 == 0:
            return 0
        return total_cut / (n*(n-1)/2)

    # ----------------------------
    # 4. Expansion
    # ----------------------------
    def expansion(self):
        expansions = []
        for community in self.communities:
            cut_edges = nx.algorithms.cut_size(self.graph, community, weight='weight')
            expansions.append(cut_edges / len(community))
        return np.mean(expansions)

    # ----------------------------
    # 5. Flake-ODF
    # ----------------------------
    def flake_odf(self):
        flakes = []
        for community in self.communities:
            for node in community:
                out_edges = sum(1 for neighbor in self.graph.neighbors(node) if neighbor not in community)
                if self.graph.degree(node) > 0:
                    if out_edges > self.graph.degree(node)/2:
                        flakes.append(1)
                    else:
                        flakes.append(0)
        if len(flakes) == 0:
            return 0
        return np.mean(flakes)

    # ----------------------------
    # 6. Average Out-Degree Fraction (AODF)
    # ----------------------------
    def aodf(self):
        fractions = []
        for community in self.communities:
            for node in community:
                out_edges = sum(1 for neighbor in self.graph.neighbors(node) if neighbor not in community)
                deg = self.graph.degree(node)
                if deg > 0:
                    fractions.append(out_edges / deg)
        if len(fractions) == 0:
            return 0
        return np.mean(fractions)

    # ----------------------------
    # 7. Modularity Density (Densely connected communities)
    # ----------------------------
    def modularity_density(self):
        total = 0
        m = self.graph.number_of_edges()
        for community in self.communities:
            subgraph = self.graph.subgraph(community)
            lc = subgraph.number_of_edges()
            dc = len(community)*(len(community)-1)/2
            if dc > 0:
                total += lc/dc
        return total / len(self.communities)

In [38]:
metrics = CommunityMetrics(G, partition)

print("Modularity:", round(metrics.modularity(),3))
print("Conductance:", round(metrics.conductance(),3))
print("Cut Ratio:", round(metrics.cut_ratio(),3))
print("Expansion:", round(metrics.expansion(),3))
print("Flake ODF:", (metrics.flake_odf()))
print("AODF:", round(metrics.aodf(),3))
print("Modularity Density:", round(metrics.modularity_density(),3))

Modularity: 0.778
Conductance: 0.071
Cut Ratio: 0.005
Expansion: 0.121
Flake ODF: 0.0
AODF: 0.034
Modularity Density: 0.471
